In [2]:
%run train_model.py -h

train_model.py

Trains RandomForestClassifier model on <input> file located in
 ../../data/processed. Saves model to ../../models.

Usage:
    train_model.py <filename> [options]

Options
    -i --input <file>       Filename, with extension, but without path
                            (../../data/processed). [default: train_df.feather]
    -o --output <file>      Filename for resulting model, without extension.
                            Will be save to ../../models/. [default: rf_model]
    -h --help               Show docstring.
    -t                      Test mode.


In [ ]:
%run train_model.py

## Predict

In [5]:
%run predict_model.py -h

predict_model.py

Predict label on '../../data/processed/<input>' data using
'../../models/<model>' model. Note defaults below. Saves predictions
to a pickled DataFrame.

Usage:
    predict_model.py [options]

Options:
    -h --help                   Show doctsring.
    -i <file>, --input <file>   Input (test_df) filename with file extension but without
                                path (../../data/processed) [default: test_df.feather].
    -m <file>, --model <file>   Model filename without path (../../models/) or file extension
                                [default: rf_model]
    -o <file>, --output <file>  Output filename without path (../../data/processed/)
                                [default: test_pred.feather]
    -t --test                   Test mode. [default: False]


In [1]:
%run predict_model.py

INFO:root:test data: ../../data/processed/test_df.feather
INFO:root:Model: ../../models/rf_model
DEBUG:root:Loading data & model...
DEBUG:root:Predicting...
DEBUG:root:Saving predictions...


Fitting score:  0.2015847860538827


DEBUG:root:Saving...
INFO:root:Output (predictions) saved to: ../../data/processed/test_pred.feather
INFO:root:Script complete!


Elapsed time: 1 minutes


In [2]:
%run predict_model.py -i train_df.feather -o train_pred.feather

INFO:root:test data: ../../data/processed/train_df.feather
INFO:root:model: ../../models/rf_model
DEBUG:root:Loading data & model...
DEBUG:root:Predicting...
DEBUG:root:Saving predictions...


Fitting score:  0.21961687428033078


DEBUG:root:Saving...
INFO:root:Output (predictions) saved to: ../../data/processed/tmp.feather
INFO:root:Script complete!


Elapsed time: 4 minutes
